In [2]:
import numpy as np
import pandas as pd
import sys
from pathlib import Path
sys.path.append('..') 
import platform
import os
from pathlib import Path
import shutil
from pathlib import Path
import shutil
from utils.behavior.session_utils import load_session_df, parse_session_string
from utils.behavior.lick_analysis import clean_up_licks, parse_lick_trains
from scipy.io import loadmat
from itertools import chain
from matplotlib import pyplot as plt
from IPython.display import display
from scipy.signal import find_peaks
from harp.clock import align_timestamps_to_anchor_points
from utils.basics.data_org import *
from utils.photometry.preprocessing import * 
from utils.photometry.plot_utils import align_signal_to_events, color_gradient, plot_FP_with_licks, plot_G_vs_Iso, plot_FP_beh_analysis
import numpy as np
from scipy.signal import butter, filtfilt, medfilt, sosfiltfilt
from scipy.optimize import curve_fit
import json
from sklearn.linear_model import LinearRegression
from matplotlib.gridspec import GridSpec
import pickle
from aind_fip_dff.utils.preprocess import batch_processing, tc_triexpfit
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
import time
from contextlib import redirect_stdout
# %matplotlib inline
# %matplotlib widget

In [4]:
session = 'm699461d20240105'
session_dirs = parse_session_string(session)

In [5]:
session_dirs

{'aniName': '699461',
 'suptitleName': 'm699461d20240105',
 'sessionFolder': 'm699461d20240105',
 'sortedFolder': 'F:\\\\699461\\m699461d20240105\\sorted\\session',
 'animalName': '699461',
 'saveFigName': 'm699461d20240105',
 'saveFigFolder': 'F:\\\\699461\\m699461d20240105\\figures',
 'baseFolder': 'F:\\\\699461\\m699461d20240105',
 'behavioralDataPath': 'F:\\\\699461\\m699461d20240105\\behavior\\m699461d20240105.asc',
 'date': '20240105',
 'videopath': 'F:\\\\699461\\m699461d20240105\\pupil',
 'lickPath': 'F:\\\\699461\\m699461d20240105\\lick\\session',
 'photometryPath': 'F:\\\\699461\\m699461d20240105\\photometry'}

In [6]:
import numpy as np
from scipy.special import expit  # Sigmoid function for logistic transformation

def qLearning_model_5params(start_values, choice, outcome):
    alphaNPE = start_values[0]
    alphaPPE = start_values[1]
    alphaForget = start_values[2]
    beta = start_values[3]
    bias = start_values[4]

    trials = len(choice)
    Q = np.zeros((trials, 2))
    pe = np.zeros(trials)

    # Call learning rule
    for t in range(trials - 1):
        if choice[t] == 1:  # Right choice
            Q[t + 1, 0] = alphaForget * Q[t, 0]
            pe[t] = outcome[t] - Q[t, 1]
            Q[t + 1, 1] = Q[t, 1] + (alphaNPE if pe[t] < 0 else alphaPPE) * pe[t]
        else:  # Left choice
            Q[t + 1, 1] = alphaForget * Q[t, 1]
            pe[t] = outcome[t] - Q[t, 0]
            Q[t + 1, 0] = Q[t, 0] + (alphaNPE if pe[t] < 0 else alphaPPE) * pe[t]

    # Compute last prediction error
    pe[-1] = outcome[-1] - Q[-1, 1] if choice[-1] == 1 else outcome[-1] - Q[-1, 0]

    # Softmax rule (logistic function)
    probChoice = expit(beta * (Q[:, 1] - Q[:, 0]) + bias)

    # Calculate likelihood
    LH = likelihood(choice, probChoice)

    # Compute chosen probabilities
    probChosen = np.where(choice == 0, 1 - probChoice, probChoice)

    return LH, probChosen, Q, pe

def q_learning_model_5params_k(start_values, choice, outcome):
    alpha = start_values[0]
    aF = start_values[1]
    beta = start_values[2]
    k = start_values[3]
    bias = start_values[4]

    trials = len(choice)
    Q = np.zeros((trials, 2))
    kChoice = np.zeros(trials)
    pe = np.zeros(trials)

    # Call learning rule
    for t in range(trials - 1):
        if choice[t] == 1:  # Right choice
            Q[t + 1, 0] = aF * Q[t, 0]
            pe[t] = outcome[t] - Q[t, 1]
            Q[t + 1, 1] = Q[t, 1] + alpha * pe[t]
            kChoice[t + 1] = k
        else:  # Left choice
            Q[t + 1, 1] = aF * Q[t, 1]
            pe[t] = outcome[t] - Q[t, 0]
            Q[t + 1, 0] = Q[t, 0] + alpha * pe[t]
            kChoice[t + 1] = -k

    # Compute last prediction error
    pe[-1] = outcome[-1] - Q[-1, 1] if choice[-1] == 1 else outcome[-1] - Q[-1, 0]

    # Softmax rule (logistic function)
    probChoice = expit(beta * (Q[:, 1] - Q[:, 0]) + kChoice + bias)

    # Calculate likelihood
    LH = likelihood(choice, probChoice)

    # Compute chosen probabilities
    probChosen = np.where(choice == 0, 1 - probChoice, probChoice)

    return LH, probChosen, Q, pe

def likelihood(choice, probChoice):
    """Computes likelihood given choices and probabilities."""
    return np.sum(np.log(probChoice[choice == 1])) + np.sum(np.log(1 - probChoice[choice == 0]))


In [ ]:
def get_model_variables_dF(model_name, params, choice, outcome, laser=None):
    t = {}

    model_functions = {
        "5params": qLearning_model_5params,
        "5params_k_bias": qLearningModel_5params_k,
        "5paramsLaserNegRPE": qLearningModel_5paramsLaserNegRPE,
        "5paramsLaserNegRPERotation": qLearningModel_5paramsLaserNegRPERotation_simNoPlot,
    }
    

    # Call the appropriate function if it exists
    if model_name in model_functions:
        if "LaserNegRPE" in model_name:
            t["LH"], t["probChoice"], t["Q"], t["pe"], t["peChange"] = model_functions[model_name](params, choice, outcome, laser)
        elif "fiveParam_ph_bias" in model_name or "sixParam_ph_bias" in model_name:
            t["LH"], t["probChoice"], t["Q"], t["pe"], t["alpha"] = model_functions[model_name](params, choice, outcome)
        else:
            t["LH"], t["probChoice"], t["Q"], t["pe"] = model_functions[model_name](params, choice, outcome)
    else:
        raise ValueError(f"Unknown model name: {model_name}")

    return t
